In [1]:
from pytorch_pae import AE, utils
import optuna
from optuna.trial import TrialState
import logging
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

import torch
from torchsummary import summary
print(torch.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

study_folder  = '/global/cscratch1/sd/vboehm/OptunaStudies/'
study_name    = "SDSS_conv_AE_optimization"  
study_name    = os.path.join(study_folder, study_name)
storage_name  = "sqlite:///{}.db".format(study_name)
EPOCHS        = 15
NUM_HOURS     = 4
N_TRIALS      = 100
SEED          = 314159




# def objective(trial):
    
    
#     ## data parameters
#     dataset       = 'SDSS_DR16'
#     loc           = '/global/cscratch1/sd/vboehm/Datasets'

#     # padding values in each conv layer
#     strides      = [1]
#     paddings     = [0]
#     # stride values in each conv layer
#     # whether tp apply a layer normalization after conv layer
#     #layer_norm   = [True]
#     # whether to train elemntwise affine parameters for normalization layer 
#     affine       = False
#     # whether to Lipschitz regularize by bounding the spectral norm 
#     spec_norm    = True
#     # activation function after each layer
#     activations  = ['ReLU']
#     # whether to add a bias in each layer or not
#     bias         = [True]

#     # data dimensionality
#     dim          = '1D'
#     # number of channels in data
#     input_c      = 1 
#     # data dimensioality along one axis (only square data supported in 2D)
#     input_dim    = 1000
#     # type of encoder and decoder network (either 'fc' or 'conv')
#     encoder_type = 'conv'
#     decoder_type = 'conv'

#     # if True, the output is fed through a sigmoid layer to bring data values into range [0,1]
#     final_sigmoid = False

#     ## Training parameters
#     nepochs       = EPOCHS
#     batchsize     = 32
#     initial_lr    = 1e-2

#     optimizer        = 'Adam'
#     criterion1       = 'MSELoss'
#     criterion2       = 'masked_chi2'

#     scheduler        = 'ExponentialLR'
#     scheduler_params = {'gamma':0.95}
#     ann_epoch        = 5
    
#     n_layers   = trial.suggest_int('n_layers',1,6)
#     latent_dim = trial.suggest_int('latent_dim',8,12)
    
#     activations = activations*n_layers
#     bias        = bias*n_layers
#     #layer_norm  = layer_norm*n_layers
#     strides     = strides*n_layers
#     paddings    = paddings*n_layers
    
    
#     out_channels = []
#     kernel_sizes = []
#     scale_facs   = []
#     dropout_rate = []
#     layer_norm   = []
#     #paddings     = []
    
#     current_size = input_dim 
#     for ii in range(n_layers):
#         out_channels.append(trial.suggest_int('out_channel_%d'%ii,8,128))
#         kernel_sizes.append(trial.suggest_int('kernel_size_%d'%ii,3,max(3,int(current_size)//20)))
#         #paddings.append(trial.suggest_int('padding_%d'%ii,1,kernel_sizes[ii]//2))
#         current_size = utils.output_shape(current_size,strides[ii],paddings[ii],kernel_sizes[ii],dilation=1)
#         layer_norm.append(trial.suggest_categorical('layer_norm_%d'%ii,[True,False]))
#         if current_size>8*latent_dim:
#             scale_facs.append(trial.suggest_int('scale_fac_%d'%ii,1,4))
#         else:
#             scale_facs.append(1)
#         current_size = current_size//scale_facs[ii]
            
#         #dropout_rate.append(trial.suggest_float('dropout_rate_%d'%ii,1e-3,1,log=True))

#     print(kernel_sizes,paddings,strides)
    
#     general_params      = {'input_c': input_c, 'input_dim': input_dim, 'latent_dim': latent_dim, 'encoder_type': encoder_type, 'decoder_type': decoder_type, 'dim': dim}
#     conv_network_params = {'n_layers': n_layers, 'out_channels': out_channels, 'kernel_sizes': kernel_sizes, 'scale_facs': scale_facs, 'paddings': paddings,                       'strides': strides,'activations': activations, 'spec_norm': spec_norm, 'layer_norm': layer_norm,                       'affine': affine,'final_sigmoid': final_sigmoid, 'bias':bias}

#     training_params     = {'batchsize': batchsize, 'initial_lr': initial_lr, 'optimizer': optimizer, 'criterion1': criterion1, 'criterion2': criterion2,
#                            'scheduler': scheduler, 'scheduler_params':scheduler_params, 'ann_epoch': ann_epoch}
#     data_params         = {'dataset':dataset, 'loc': loc}
    
#     AE1                 = AE.Autoencoder(general_params,data_params,conv_network_params, conv_network_params, training_params, device)

#     try:
#         if dim =='1D':
#             summary(AE1, (input_c,input_dim))
#         else:
#             summary(AE1, (input_c, input_dim, input_dim))
        
#         train_loss, valid_loss = AE1.train(nepochs)
#     except:
#         valid_loss=[100]
        
    
#     return np.sqrt(valid_loss[-1])


# In[17]:


time = NUM_HOURS*60*60-600
study = optuna.create_study(direction='minimize',study_name=study_name, storage=storage_name,load_if_exists=True,  sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))
#study.optimize(objective, n_trials=N_TRIALS, timeout=time)


# In[18]:


pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

1.8.0
cuda:0


[I 2022-01-28 10:24:02,367] Using an existing study with name '/global/cscratch1/sd/vboehm/OptunaStudies/SDSS_conv_AE_optimization' instead of creating a new one.


Using an existing study with name '/global/cscratch1/sd/vboehm/OptunaStudies/SDSS_conv_AE_optimization' instead of creating a new one.
Study statistics: 
  Number of finished trials:  71
  Number of pruned trials:  0
  Number of complete trials:  70
Best trial:
  Value:  1.1444383394722513
  Params: 
    kernel_size_0: 44
    latent_dim: 10
    layer_norm_0: False
    n_layers: 1
    out_channel_0: 32
    scale_fac_0: 1


In [4]:
1000/20

50.0